In [119]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from builtins import range, input

# Variável para ajudar a definir o total de estados possíveis
LENGTH = 5

In [285]:
class Environment:

    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))

        self.ended = False
        self.num_states = 3**(LENGTH*LENGTH)

        self.apple = []
        self.snake = [] 

    def reset(self):
        
        self.apple = [int(np.random.rand()*(LENGTH)), int(np.random.rand()*(LENGTH))]
        self.snake = [[int(np.random.rand()*(LENGTH)), int(np.random.rand()*(LENGTH))]]
        
        print(self.apple)
        print(self.snake)
        
    def draw_board(self):
        for i in range(LENGTH):
            for j in range(LENGTH):
                p = False   
                if(j == 0):
                    print("|", end="")
                if(i == self.apple[0] and j == self.apple[1]):
                    print(" 0 ", end="") 
                    p = True
                for x, s in enumerate(self.snake):
                    if(i == s[0] and j == s[1]):
                        if(x == 0):
                            print(" > ", end="")
                            p = True
                        else:
                            print(" - ", end="")
                            p = True
                if(p == False):
                    print("   ", end="")
                if(j == LENGTH - 1):
                    print("|")
        print("")

In [286]:
env = Environment()

In [287]:
env.reset()

[4, 0]
[[1, 2]]


In [288]:
env.draw_board()

|               |
|       >       |
|               |
|               |
| 0             |

